# Sandbox env. for developing middleware classes

A generic dataclass representing a bet on an event

In [28]:
from dataclasses import dataclass
from datetime import datetime
from typing import Set

@dataclass
class Bet:
    """
    Represents a betting event.

    Attributes:
        event_id (int): The unique identifier of the event.
        bookmaker_id (int): The unique identifier of the bookmaker.
        sport (str): The sport of the betting event.
        event_date (datetime): The date and time of the event.
        event_date (datetime): The timezone in which the event occurs.
        participants (set): The set of participants in the event.
        outcome (str): The selected outcome of the bet for the target participant.
        target (int): The participant to which the chosen outcome refers.
        stake (float): The amount wagered on the bet.
        odds (float): The odds of the bet.
        odds_unit (str): The selected odds standard, ie. fractional, american, or decimal.
    """

    event_id: int
    bookmaker_id: int
    sport: str
    event_date: datetime
    event_tz: str
    participants: Set[str]
    outcome: str
    target: str
    stake: float
    odds: float
    odds_unit: str


    def __post_init__(self):
        """
        Additional validation for the Bet object instance attributes.

        Raises:
            ValueError: If the target participant is not in the participants set.
            ValueError: If the event_tz string is not a valid timezone.
        """
        if self.target not in self.participants:
            raise ValueError("Target participant must be one of the participants")
        
        # You can add more checks for other attributes here
        
        # Check if event_tz is a valid timezone
        from pytz import all_timezones
        if self.event_tz not in all_timezones:
            raise ValueError("Invalid timezone")

In [17]:
import pytz

for tz in pytz.all_timezones:
    print(tz)


Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara
Africa/Asmera
Africa/Bamako
Africa/Bangui
Africa/Banjul
Africa/Bissau
Africa/Blantyre
Africa/Brazzaville
Africa/Bujumbura
Africa/Cairo
Africa/Casablanca
Africa/Ceuta
Africa/Conakry
Africa/Dakar
Africa/Dar_es_Salaam
Africa/Djibouti
Africa/Douala
Africa/El_Aaiun
Africa/Freetown
Africa/Gaborone
Africa/Harare
Africa/Johannesburg
Africa/Juba
Africa/Kampala
Africa/Khartoum
Africa/Kigali
Africa/Kinshasa
Africa/Lagos
Africa/Libreville
Africa/Lome
Africa/Luanda
Africa/Lubumbashi
Africa/Lusaka
Africa/Malabo
Africa/Maputo
Africa/Maseru
Africa/Mbabane
Africa/Mogadishu
Africa/Monrovia
Africa/Nairobi
Africa/Ndjamena
Africa/Niamey
Africa/Nouakchott
Africa/Ouagadougou
Africa/Porto-Novo
Africa/Sao_Tome
Africa/Timbuktu
Africa/Tripoli
Africa/Tunis
Africa/Windhoek
America/Adak
America/Anchorage
America/Anguilla
America/Antigua
America/Araguaina
America/Argentina/Buenos_Aires
America/Argentina/Catamarca
America/Argentina/ComodRivad

In [19]:
import pytz

# Create a string representing the datetime
datetime_str = '2021-06-01 12:00:00'

# Parse the string to a naive datetime object
naive_datetime = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')

# Create a timezone-aware datetime object
timezone = pytz.timezone('Australia/Melbourne')
aware_datetime = timezone.localize(naive_datetime)

# Print the timezone-aware datetime object
print(aware_datetime)


2021-06-01 12:00:00+10:00


In [29]:
Bet(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Perth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Silly Goose',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal',
)

Bet(event_id=1, bookmaker_id=1, sport='duck duck goose', event_date=datetime.datetime(2021, 6, 1, 12, 0), event_tz='Australia/Perth', participants={'Daffy Duck', 'Silly Goose'}, outcome='win', target='Silly Goose', stake=99.0, odds=1.4, odds_unit='decimal')

Test validations

In [30]:
Bet(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Perth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Stupid bird',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal',
)

ValueError: Target participant must be one of the participants

In [31]:
Bet(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Parth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Silly Goose',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal',
)

ValueError: Invalid timezone

In [32]:
Bet(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = '2021-06-01 12:00:00',
    event_tz = "Australia/Perth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Silly Goose',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal',
)

Bet(event_id=1, bookmaker_id=1, sport='duck duck goose', event_date='2021-06-01 12:00:00', event_tz='Australia/Perth', participants={'Daffy Duck', 'Silly Goose'}, outcome='win', target='Silly Goose', stake=99.0, odds=1.4, odds_unit='decimal')